In [1]:
# Dependencies
import matplotlib.pyplot as plt
%matplotlib inline

import os
import numpy as np
import tensorflow as tf
from keras.models import load_model

from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import Hyperband

from tensorflow.keras.preprocessing.image import ImageDataGenerator


2024-03-10 14:02:46.913294: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Set image and batch size
image_size = (224,224)
batch_size = 100

# Define the directories for training and testing data
train_data = 'train/'
test_data = 'test/'

# Create the image generator for normilization
train_data_gen = ImageDataGenerator(horizontal_flip=True)
test_data_gen = ImageDataGenerator(horizontal_flip=True)

# Generate the batches of training and test data
train_generator = train_data_gen.flow_from_directory(
    train_data,
    target_size = image_size,
    batch_size = batch_size,
    color_mode = 'grayscale',
    class_mode = 'categorical'
)

test_generator = test_data_gen.flow_from_directory(
    test_data,
    target_size = image_size,
    batch_size = batch_size,
    color_mode = 'grayscale',
    class_mode = 'categorical'
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [6]:
# Define a function to build the model
def create_model(hp):
    model = keras.Sequential()
    model.add(layers.Conv2D(hp.Int('conv1_units', min_value=32, max_value=256, step=32), (3, 3), activation='relu', input_shape=(224, 224, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(hp.Int('conv2_units', min_value=32, max_value=256, step=32), (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(hp.Int('conv3_units', min_value=32, max_value=256, step=32), (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(hp.Int('dense_units', min_value=32, max_value=256, step=32), activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model




In [7]:
tuner = Hyperband(
    create_model,
    objective='val_accuracy',
    max_epochs=30,
    directory='tuner',
    project_name='emotion_detection_test',
    overwrite=False)



Reloading Tuner from tuner/emotion_detection_test/tuner0.json


In [9]:
tuner.search(train_generator, validation_data=test_generator)


Search: Running Trial #5

Value             |Best Value So Far |Hyperparameter
192               |32                |conv1_units
256               |224               |conv2_units
64                |128               |conv3_units
96                |64                |dense_units
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
3                 |3                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
  2/288 [..............................] - ETA: 34:07:53 - loss: 138.0024 - accuracy: 0.5000

In [8]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.save('output/best_model_optimized.h5')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [2]:
# Load the trained model
model = load_model('output/best_model_optimized.h5')

In [5]:
# Evaluate the model
loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test accuracy: {accuracy}')

71/71 [==============================] - 808s 11s/step - loss: 0.4165 - accuracy: 0.8571
Test accuracy: 0.857143223285675


In [6]:
from sklearn.metrics import classification_report

# Generate predictions for the test data
y_pred = model.predict(test_generator)

# Convert the predicted probabilities to class labels
y_pred_classes = np.argmax(y_pred, axis=1)

# Get the true class labels from the test data generator
y_true = test_generator.classes

# Compute precision, recall, and F1-score
report = classification_report(y_true, y_pred_classes, target_names=test_generator.class_indices.keys())

# Print the classification report
print(report)

72/72 [==============================] - 709s 10s/step
              precision    recall  f1-score   support

       angry       0.13      1.00      0.24       958
     disgust       0.00      0.00      0.00       111
        fear       0.00      0.00      0.00      1024
       happy       0.00      0.00      0.00      1774
     neutral       0.00      0.00      0.00      1233
         sad       0.00      0.00      0.00      1247
    surprise       0.00      0.00      0.00       831

    accuracy                           0.13      7178
   macro avg       0.02      0.14      0.03      7178
weighted avg       0.02      0.13      0.03      7178



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  